In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import os

In [2]:
data_dir = "/app/bucket/data/raw_data"

In [67]:
def create_lag_feature(df, target, n_lags, freq, extend_rows=False, drop_target=True):
    """
    DataFrame의 특정 열(들)에 대해 지정된 기간만큼 Lag 변수를 생성합니다.

    Args:
        df (pd.DataFrame): 시계열 인덱스(DatetimeIndex)를 가진 원본 데이터프레임.
        target (str): Lag 변수를 생성할 열 이름.
        n_lags (int): 지연시킬 기간 (정수 시점 또는 시간 문자열).
            - int: 시계열 인덱스의 '행' 개수만큼 지연 (예: 1일, 7일).
        freq (str): df의 인덱스(datetime)의 빈도 문자열 (예: 'D', 'M', 'MS').
        extend_rows (bool, optional): True이면 데이터프레임의 마지막 날짜 이후로 n_lags 기간만큼의 새로운 인덱스를 추가합니다. 기본값은 False입니다.
        drop_target (bool, optional): True이면 원본 target 열을 삭제합니다. 기본값은 True입니다.
    
    Returns:
        new_df (pd.DataFrame): Lag 변수가 추가된 새로운 데이터프레임.
    """
    new_df = df.copy()
    new_df = new_df.sort_index()
    index_name = new_df.index.name if new_df.index.name else 'ds'

    if extend_rows:
        # 마지막 날짜 이후로 n_lags 기간만큼의 새로운 인덱스 생성
        start = new_df.index[-1] + pd.tseries.frequencies.to_offset(freq)
        future_dates = pd.date_range(start=start, periods=n_lags, freq=freq)
        new_df = pd.concat([new_df, pd.DataFrame(index=future_dates)], axis=0)
        new_df.index.name = index_name
        
    # Lag 변수명 생성 (예: 'Close_lag_1' 또는 'VIX_lag_2M')
    lag_name = f"{target}_lag_{n_lags}{freq}"
    # shift() 함수를 사용하여 Lag 변수 생성
    new_df[lag_name] = new_df[target].shift(periods=n_lags, freq=None)

    if drop_target:
        # 원본 target 열 삭제
        new_df.drop(columns=[target], inplace=True)

    return new_df

In [61]:
def MS_to_D_ffill(df):
    """
    월초(freq='MS') DatetimeIndex를 가진 DataFrame을 
    일별(Daily) 빈도로 확장하고, NaN 값을 ffill로 채웁니다.

    Args:
        df (pd.DataFrame): 월별 DatetimeIndex를 가진 DataFrame.
    
    Returns:
        new_df (pd.DataFrame): 일별 빈도로 확장되고 ffill 처리된 DataFrame.
    """

    new_df = df.copy()
    new_df = new_df.sort_index()
    
    # 1. 일별 DatetimeIndex 생성
    daily_index = pd.date_range(
        start=new_df.index.min(),
        end=new_df.index.max() + pd.tseries.frequencies.to_offset("MS"),
        freq='D' # 일별 빈도(Daily Frequency) 지정
    )
    
    # 2. Reindex (일별 인덱스로 확장)
    index_name = new_df.index.name if new_df.index.name else 'ds'
    new_df = new_df.reindex(daily_index)
    new_df.index.name = index_name
    
    # 3. ffill (Forward Fill)
    new_df = new_df.fillna(method='ffill')

    return new_df

In [ ]:
# SPY 데이터 읽기 및 전처리
spy_df = pd.read_csv(os.path.join(data_dir, 'spy_data.csv'))
spy_df = spy_df[['datetime', 'close', 'volume']]
spy_df.rename(columns={'close': 'spy_close', 'volume': 'spy_volume'}, inplace=True)
spy_df['ds'] = pd.to_datetime(spy_df['datetime'], format='%Y-%m-%d')
spy_df.set_index('ds', inplace=True)
spy_df.drop(columns=['datetime'], inplace=True)
spy_df = spy_df.sort_index()
spy_df = create_lag_feature(df=spy_df, target='spy_volume', n_lags=1, freq="D", extend_rows=False, drop_target=True)
spy_df.tail(10)

In [ ]:
cli_df = pd.read_csv(os.path.join(data_dir, 'cli_data.csv'))
cli_df = cli_df[['datetime', 'CLI']]
cli_df['ds'] = pd.to_datetime(cli_df['datetime'], format='%Y-%m-%d')
cli_df.set_index('ds', inplace=True)
cli_df.drop(columns=['datetime'], inplace=True)
cli_df = cli_df.sort_index()
cli_df = create_lag_feature(df=cli_df, target='CLI', n_lags=1, freq="MS", extend_rows=True, drop_target=True)
cli_df = MS_to_D_ffill(cli_df)
cli_df.tail(20)

In [ ]:
total_df = pd.merge(spy_df,
                     cli_df,
                     left_index=True,
                     right_index=True,
                     how="left")
total_df = total_df.fillna(method="ffill")
total_df = total_df.dropna(how="any")
print(total_df.shape)
total_df